In [ ]:
import xmlrpc.client 
import pandas as pd

# Conexión con Odoo
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")




fields_info = models.execute_kw(
    db, uid, password,
    'sale.order',  # Nombre técnico del modelo
    'fields_get',
    []
)

# Convertir a DataFrame para visualizar mejor
df_campos = pd.DataFrame.from_dict(fields_info, orient='index')
df_campos

# TOTAL FACTURAS 

In [ ]:
import xmlrpc.client
from datetime import date, timedelta, datetime
import pandas as pd

# Conexión con Odoo (manteniendo tus credenciales)
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos


# --- Autenticación ---
common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- Fechas ---
weeks = 56

# --- Fechas automáticas ---
hoy = date.today()
fecha_fin = hoy.strftime("%Y-%m-%d")
fecha_inicio = (hoy - timedelta(weeks=weeks)).strftime("%Y-%m-%d")

# --- 1) Buscar facturas válidas (account.invoice) ---

invoice_domain = [
    ("date_invoice", ">=", fecha_inicio),
    ("date_invoice", "<=", fecha_fin),
    ("type", "=", "out_invoice"),    # solo ventas
    ("state", "in", ["open", "paid"])
]

invoice_ids = models.execute_kw(
    db, uid, password,
    "account.invoice", "search",
    [invoice_domain]
)
print(f"Facturas encontradas: {len(invoice_ids)}")

# --- 2) Descargar las líneas de esas facturas (account.invoice.line) ---

# Campos de la LÍNEA de factura. Eliminamos 'number', 'user_id', 'section_id', 'partner_id' porque irán en la factura.
line_fields = ["product_id", "quantity", "price_subtotal", "invoice_id","create_date","origin"] 

records = []
limit = 20000
offset = 0

while True:
    result = models.execute_kw(
        db, uid, password,
        "account.invoice.line", "search_read",
        [[("invoice_id", "in", invoice_ids)]],
        {"fields": line_fields, "limit": limit, "offset": offset}
    )
    if not result:
        break
    records.extend(result)
    offset += limit
    print(f"Descargados {len(records)} registros de líneas...")

# --- 3) Pasar a DataFrame de líneas ---
line_df = pd.DataFrame(records).fillna(0)


# Separar product_id
line_df["product_id_num"] = line_df["product_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["product_name"] = line_df["product_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar invoice_id
line_df["invoice_id_num"] = line_df["invoice_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["invoice_name"] = line_df["invoice_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Convertir fecha a datetime
line_df["date_invoice"] = pd.to_datetime(line_df["create_date"], errors="coerce")

# Eliminar las columnas originales problemáticas
line_df = line_df.drop(columns=["invoice_id","create_date"])

print(f"Total de líneas descargadas: {len(line_df)}")

# ----------------------------------------------------
# --- 4) Descargar los campos adicionales de Factura (account.invoice) ---
# ----------------------------------------------------
# Añadimos los campos que quieres: number, user_id, section_id, y también partner_id y store_id
invoice_fields = ["id", "store_id", "number", "user_id", "section_id", "partner_id"]
invoices = models.execute_kw(
    db, uid, password,
    "account.invoice", "read",
    [invoice_ids], # Solo las facturas que encontramos
    {"fields": invoice_fields}
)
invoice_df = pd.DataFrame(invoices)

# --- 5) Procesar campos de la factura ---

# Separar store_id
invoice_df["store_name"] = invoice_df["store_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar user_id (Vendedor)
invoice_df["salesperson_name"] = invoice_df["user_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)

# Separar section_id (Equipo de Ventas)
invoice_df["sales_team_name"] = invoice_df["section_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)

# Separar partner_id (Cliente/Partner)
invoice_df["partner_id_num"] = invoice_df["partner_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
# El nombre del partner es el segundo elemento de la tupla (si existe)
invoice_df["client_name_inv"] = invoice_df["partner_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)


# Eliminar columnas originales no deseadas o ya procesadas
invoice_df = invoice_df.drop(columns=["store_id", "user_id", "section_id", "partner_id"])


# ----------------------------------------------------
# --- 6) Fusionar DataFrames ---
# ----------------------------------------------------

# Fusionamos las líneas de factura (line_df) con los datos de las facturas (invoice_df)
df_merged = line_df.merge(
    invoice_df, 
    left_on="invoice_id_num", 
    right_on="id", 
    how="left"
)

# Limpieza final de columnas de IDs de factura
df_merged = df_merged.drop(columns=["invoice_id_num", "product_id_num"])


df_merged['origin'] = df_merged['origin'].astype('string')
df_merged["product_id"] = df_merged["product_id"].astype(str)

# Guardar
df_merged.to_parquet(f"ventashistoricas{weeks}semanas.parquet", index=False)

print(f"Total de líneas descargadas con información de factura: {len(df_merged)}")

#para 52 semanas se demora aproximadamente 6.5 minutos
#para 56 semanas se demora aproximadamente 7 minutos
#para 105 semanas se demora aproximadamente 14 minutos

Facturas encontradas: 98514
Descargados 20000 registros de líneas...


KeyboardInterrupt: 

In [7]:
print(f"Fecha primera factura",df_merged["date_invoice"].min())
print(f"Fecha ultima factura",df_merged["date_invoice"].max())

Fecha primera factura 2024-02-23 12:03:29
Fecha ultima factura 2025-10-03 18:42:09


In [8]:
pd.set_option("display.max_colwidth", None)  # muestra toda la celda completa
df_merged.sample(5)

,origin,product_id,price_subtotal,id_x,quantity,product_name,invoice_name,date_invoice,number,id_y,store_name,salesperson_name,sales_team_name,partner_id_num,client_name_inv
233821,BD1\OUT\54857,"[17016, '[DAB02524025] DA2524 FILTRO AIRE - GMC BRIGADIER (025 DA2524)']",361200.0,1349956,3.0,[DAB02524025] DA2524 FILTRO AIRE - GMC BRIGADIER (025 DA2524),FCOT20155,2024-07-29 16:33:04,FCOT20155,367680,PRINCIPAL COTA,ANDRES CORRALES,EXTERNOS,28237,[CO9008251487] SERVICIO TECNICO THERMO KING S.A.S
55168,BD1\OUT\59118,"[19131, '[BCS00483125] GS483 FILTRO COMBUSTIBLE - MASSEY FERGUSON - VOLVO (125 BF5587D)']",61700.0,1561534,1.0,[BCS00483125] GS483 FILTRO COMBUSTIBLE - MASSEY FERGUSON - VOLVO (125 BF5587D),FCOT25120,2025-07-01 20:14:59,FCOT25120,423998,PRINCIPAL COTA,HUGO ALEJANDRO ZUBIETA FERNANDEZ,EXTERNOS,4588,[CO9005212671] FILTROS Y ACEITES LA PAZ
17085,BD5\OUT\63047,"[19237, '[BCS00229125] GS229 FILTRO COMBUSTIBLE SEPARADOR J. DEERE, PERKINS. (125 BF7673D)']",64100.0,1607489,1.0,"[BCS00229125] GS229 FILTRO COMBUSTIBLE SEPARADOR J. DEERE, PERKINS. (125 BF7673D)",FMDE3797,2025-09-05 14:05:57,FMDE3797,435999,SUCURSAL MEDELLIN,Omar Hernandez,SUCURSALES / MEDELLÍN / EXTERNOS MED,12964,[CO9002143213] INGENIERIA DE TELECOMUNICACIONES IDT S.A.S
11153,BD6/OUT/49315,"[50017, '[DCS00411189] GS411 FILTRO COMBUSTIBLE - HYUNDAI (189 GS411)']",334000.0,1614317,20.0,[DCS00411189] GS411 FILTRO COMBUSTIBLE - HYUNDAI (189 GS411),FBAQ14318,2025-09-15 20:28:52,FBAQ14318,437842,SUCURSAL BARRANQUILLA,JAIME ALBERTO CAÑON RAMIREZ,SUCURSALES / BARRANQUILLA / EXTERNOS,12174,[CO9009066343] LOGICEM SAS
114297,BD6/OUT/44387,"[19203, '[BCS00461125] GS461 FILTRO COMB. DISPENSADORES COMB. CIM-TEK 80030 (125 BF7623)']",154800.0,1490748,2.0,[BCS00461125] GS461 FILTRO COMB. DISPENSADORES COMB. CIM-TEK 80030 (125 BF7623),FBAQ12851,2025-03-14 15:09:21,FBAQ12851,405460,SUCURSAL BARRANQUILLA,JAIME ALBERTO CAÑON RAMIREZ,SUCURSALES / BARRANQUILLA / EXTERNOS,18305,[CO9011918715] DISTRIBUCIONES MARTINEZ C S.A.S


In [9]:
#df_merged[df_merged["product_name"].str.contains("DA4020", case =False , na=False)].sample(10)

# COSTO DEL PRODUCTO

In [ ]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

campos = models.execute_kw(
    db , uid , password,
    "cs.analisis.costo","fields_get",
    [],

)

df_campos = pd.DataFrame(campos)
#df_campos.info()


productos_costo = models.execute_kw(
    db , uid , password,
    "cs.analisis.costo","search_read",
    [[]],
    {"fields":["producto_costo_unitario","producto_id"]}


)

# Convertir en df
df_productos_costo = pd.DataFrame(productos_costo)


# Separar lista anidada
df_productos_costo[["product_id_num","product_name"]] = (df_productos_costo["producto_id"].apply(pd.Series))

#Agregar columna estandarizada
df_productos_costo["product_name"] = df_productos_costo["product_name"].str.strip()

#DF normalizado
df_productos_costo = df_productos_costo[["product_name","producto_costo_unitario"]]

df_productos_costo.to_parquet("costo_productos.parquet")


# ANALISIS COSTOS

In [12]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

productos_ema = models.execute_kw(
    db, uid, password,
    "cs.analisis.costo", "search_read",[],
    #[
    #    [["semana","=",36]]
    #],
    {"fields": ["sucursal_id","product_name","ano","semana","ema","producto_costo_unitario","acumulado_combinado","clasificacion","almacen_id" ]}
)

df = pd.DataFrame(productos_ema)

def extract_id(val):
    if isinstance(val, (list, tuple)) and len(val) > 0:
        return val[0]
    return None

def extract_name(val):
    if isinstance(val, (list, tuple)) and len(val) > 1:
        return val[1]
    return str(val)

df["store_id_num"] = df["sucursal_id"].apply(extract_id)
df["store_name"]   = df["sucursal_id"].apply(extract_name)

df["almacen_id_num"] = df["almacen_id"].apply(extract_id)
df["almacen_name"]   = df["almacen_id"].apply(extract_name)

df.to_csv("real_ema.csv")

#timpo aprox  1 minutos

# VENTAS PERDIDAS

In [2]:
import xmlrpc.client
from datetime import date, timedelta
import pandas as pd

# ===============================
# 1. Conexión con Odoo
# ===============================

ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Definir rango de semanas
# ===============================

# Número de semanas que quieres descargar (puedes cambiarlo)
NUM_SEMANAS = 52  

# Fecha de hoy
hoy = date.today()
# Fecha de inicio según número de semanas atrás
fecha_inicio = hoy - timedelta(weeks=NUM_SEMANAS)

FECHA_INICIO = fecha_inicio.strftime("%Y-%m-%d")
FECHA_FIN = hoy.strftime("%Y-%m-%d")

print(f"📅 Descargando registros desde {FECHA_INICIO} hasta {FECHA_FIN}")

# ===============================
# 3. Leer ventas perdidas
# ===============================

lost_sales_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "ventas.perdidas", "search",
    [[["fecha", ">=", FECHA_INICIO], ["fecha", "<=", FECHA_FIN], ["almacenamiento_tipo", "=", "agotado"]]]
)

lost_sales_data = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "ventas.perdidas", "read",
    [lost_sales_ids],
    {"fields": ["origen", "store_id", "cliente_id", "fecha", "product_ref",
                "cantidad", "cantidad_existencia", "cantidad_reservada",
                "almacenamiento_tipo"]}
)

df = pd.DataFrame(lost_sales_data)

# ===============================
# 4. Normalizar columnas
# ===============================

for col in ["store_id", "cliente_id"]:
    df[col.replace('_id', '_name')] = df[col].apply(
        lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else str(x) if x else None
    )
    df[col + "_num"] = df[col].apply(
        lambda x: x[0] if isinstance(x, (list, tuple)) and len(x) > 0 else None
    )

# Eliminar columnas originales
df = df.drop(columns=["id", "store_id", "cliente_id"], errors='ignore')

# ===============================
# 4.1. Forzar tipos de columnas antes de exportar
# ===============================
for col in df.columns:
    if col.endswith("_num"):
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
    else:
        df[col] = df[col].astype("string")

print(df.dtypes)


# ===============================
# 5. Exportar a Parquet con nombre dinámico
# ===============================

file_name = f"ventas_perdidas_{FECHA_INICIO}_a_{FECHA_FIN}.parquet"

try:
    df.to_parquet(file_name, index=False, engine="pyarrow", compression="snappy")
    print(f"✅ Exportación exitosa con pyarrow: {file_name}")
except ImportError as e:
    print("⚠️ Módulo pyarrow no encontrado, probando con fastparquet:", e)
    try:
        df.to_parquet(file_name, index=False, engine="fastparquet", compression="snappy")
        print(f"✅ Exportación exitosa con fastparquet: {file_name}")
    except ImportError as e:
        print("❌ Instala 'pyarrow' o 'fastparquet' para exportar.")

print("\n")
print(f"El proceso ha terminado. Se extrajeron {len(df)} registros de ventas perdidas.")
print("Las columnas del DataFrame final son:", list(df.columns))



    ##TARDA MAS O MENOS 1 MINUTOS TODO 2025


📅 Descargando registros desde 2024-10-02 hasta 2025-10-01
cantidad_reservada     string[python]
product_ref            string[python]
cantidad               string[python]
almacenamiento_tipo    string[python]
fecha                  string[python]
cantidad_existencia    string[python]
origen                 string[python]
store_name             string[python]
store_id_num                    Int64
cliente_name           string[python]
cliente_id_num                  Int64
dtype: object
✅ Exportación exitosa con pyarrow: ventas_perdidas_2024-10-02_a_2025-10-01.parquet


El proceso ha terminado. Se extrajeron 68549 registros de ventas perdidas.
Las columnas del DataFrame final son: ['cantidad_reservada', 'product_ref', 'cantidad', 'almacenamiento_tipo', 'fecha', 'cantidad_existencia', 'origen', 'store_name', 'store_id_num', 'cliente_name', 'cliente_id_num']


# MOTOR BUSQUEDA

In [1]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# ===============================
# Ver campos disponible
# ===============================
fields = models.execute_kw(
    db, uid, password,
    "account.invoice.report", "fields_get",  # #mrp.report.verificacion.fuerza.lab
    []
)

for field, info in sorted(fields.items(), key=lambda x: x[1].get("string", "")):
    print(f"{field:40} | {info.get('type')} | {info.get('string')}")



nbr                                      | integer | # of Invoices
account_id                               | many2one | Account
account_line_id                          | many2one | Account Line
user_currency_price_average              | float | Average Price
price_average                            | float | Average Price
partner_bank_id                          | many2one | Bank Account
categ_id                                 | many2one | Category of Product
state_id                                 | many2one | Ciudad
product_class                            | many2one | Clase
company_id                               | many2one | Company
country_id                               | many2one | Country of the Partner Company
currency_id                              | many2one | Currency
currency_rate                            | float | Currency Rate
date                                     | date | Date
display_name                             | char | Display Name
date_due           

In [3]:
import xmlrpc.client

# ==================================
# Configuración de conexión a Odoo
# ==================================
url = "https://donsson.com"
db = "Donsson_produccion"
username = "juan.cano@donsson.com"
password = "1000285668"

# ==================================
# Conexión y autenticación
# ==================================
common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# ==================================
# Buscar modelos que contengan "report" o "sql"
# ==================================
posibles = models.execute_kw(
    db, uid, password,
    "ir.model", "search_read",
    [[["model", "ilike", "report"]]],
    {"fields": ["model", "name"], "limit": 100}
)

print("📋 Modelos encontrados con 'report' o 'sql':\n")
for m in posibles:
    print(f"{m['model']:40} | {m['name']}")


📋 Modelos encontrados con 'report' o 'sql':

account.analytic.cost.ledger.journal.report | Account Analytic Cost Ledger For Journal Report
account.analytic.journal.report          | Account Analytic Journal
account.balance.report                   | Trial Balance Report
account.budget.crossvered.report         | Account Budget crossvered report
account.budget.crossvered.summary.report | Account Budget  crossvered summary report
account.budget.report                    | Account Budget report for analytic account
account.common.account.report            | Account Common Account Report
account.common.journal.report            | Account Common Journal Report
account.common.partner.report            | Account Common Partner Report
account.common.report                    | Account Common Report
account.entries.report                   | Journal Items Analysis
account.financial.report                 | Account Report
accounting.report                        | Accounting Report
account.invoi

# INVENTARIO

In [7]:
import pandas as pd
import requests
from datetime import datetime
import os

# ==============================
# 🔧 CONFIGURACIÓN
# ==============================
url = "https://www.donsson.com/report_sql/static/inventario_bodega.xlsx"

# Carpeta donde guardar el histórico (se crea si no existe)
output_dir = "reportes_inventario"
os.makedirs(output_dir, exist_ok=True)

# Fecha y hora actual para trazabilidad
fecha_hora = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Nombre del archivo con fecha y hora
nombre_archivo = f"inventario_bodega_{fecha_hora}.xlsx"
ruta_archivo = os.path.join(output_dir, nombre_archivo)

# ==============================
# 📥 DESCARGAR REPORTE
# ==============================
print("Descargando reporte desde Odoo...")

response = requests.get(url)
if response.status_code == 200:
    with open(ruta_archivo, "wb") as f:
        f.write(response.content)
    print(f"✅ Reporte descargado y guardado como: {ruta_archivo}")
else:
    raise Exception(f"❌ Error al descargar el reporte (status {response.status_code})")

# ==============================
# 📊 LEER EN DATAFRAME
# ==============================
try:
    df = pd.read_excel(ruta_archivo)
    print(f"✅ Archivo leído correctamente ({len(df):,} filas)")
except Exception as e:
    raise Exception(f"⚠️ Error al leer el Excel: {e}")

# ==============================
# 🪶 Mostrar ejemplo de datos
# ==============================
print(df.head())

# (Opcional) Guardar también en formato CSV si quieres trazabilidad adicional
csv_ruta = ruta_archivo.replace(".xlsx", ".csv")
df.to_csv(csv_ruta, index=False)
print(f"📦 Copia CSV creada: {csv_ruta}")


Descargando reporte desde Odoo...
✅ Reporte descargado y guardado como: reportes_inventario/inventario_bodega_2025-10-17_12-13-36.xlsx
✅ Archivo leído correctamente (15,254 filas)
   Unnamed: 0  Template                       Linea  \
0           0      7997  Master Aire Elemento Axial   
1           1      7998      Master Aceite Elemento   
2           2      7999      Master Aceite Elemento   
3           3      8003         Master Aire Redondo   
4           4      8003         Master Aire Redondo   

                                      Cuenta   Referencia  \
0  1435050712-COMPRAS FILTROS COMERCIALIZADO  DAE04207004   
1  1435050712-COMPRAS FILTROS COMERCIALIZADO  DLE00314004   
2  1435050712-COMPRAS FILTROS COMERCIALIZADO  DLE00401004   
3  1435050712-COMPRAS FILTROS COMERCIALIZADO  DAE05128004   
4  1435050712-COMPRAS FILTROS COMERCIALIZADO  DAE05128004   

                                 Producto       Variante Clase  \
0    DA4207 FILTRO AIRE SEGURIDAD KOMATSU    004 AF574SY

In [3]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# 1) Traer nóminas (slips) solo de 2025
# ========================================
slips = models.execute_kw(
    db, uid, password,
    "hr.payslip", "search_read",
    [[["date_from", ">=", "2024-01-01"], ["date_to", "<=", "2025-12-31"]]],
    {"fields": ["id", "employee_id", "name", "date_from", "date_to"]}
)
df_slips = pd.DataFrame(slips)


# ========================================
# 2) Traer líneas de nómina (solo de los slips obtenidos)
# ========================================
slip_ids = df_slips["id"].tolist()
lines = models.execute_kw(
    db, uid, password,
    "hr.payslip.line", "search_read",
    [[["slip_id", "in", slip_ids]]],
    {"fields": ["id", "slip_id", "employee_id", "salary_rule_id", "total"]}
)
df_lines = pd.DataFrame(lines)

# ===============================
# 3) Expandir campos Many2one (id, name)
# ===============================
def extract_name(val):
    if isinstance(val, list) and len(val) == 2:
        return val[1]
    return None

def extract_id(val):
    if isinstance(val, list) and len(val) == 2:
        return val[0]
    return None

df_lines["employee_name"] = df_lines["employee_id"].apply(extract_name)
df_lines["slip_name"]     = df_lines["slip_id"].apply(extract_name)
df_lines["rule_name"]     = df_lines["salary_rule_id"].apply(extract_name)

# 🔑 Extraer ID de slip_id para poder hacer el merge
df_lines["slip_id_only"] = df_lines["slip_id"].apply(extract_id)

# ===============================
# 4) Unir líneas con fechas de slip
# ===============================
df_slips["employee_name"] = df_slips["employee_id"].apply(extract_name)

df = pd.merge(
    df_lines,
    df_slips[["id", "employee_name", "date_from", "date_to"]],
    left_on="slip_id_only",
    right_on="id",
    how="left",
    suffixes=("", "_slip")
)

# ===============================
# 5) Filtrar reglas relevantes
# ===============================
df_filtro = df[df["rule_name"].isin(
    ["Salario Básico", "Salario Mínimo", "Salario Neto"]
)].copy()

# ===============================
# 6) Agregar columna período (Año-Mes)
# ===============================
df_filtro["periodo"] = pd.to_datetime(df_filtro["date_from"]).dt.to_period("M")

# ===============================
# 7) Pivotear para resumir salarios
# ===============================
tabla = df_filtro.pivot_table(
    index=["employee_name", "periodo"],
    columns="rule_name",
    values="total",
    aggfunc="sum",
    fill_value=0
).reset_index()

# ===============================
# 8) Renombrar columnas
# ===============================
tabla = tabla.rename(columns={
    "Salario Básico": "salario_basico",
    "Salario Mínimo": "salario_minimo",
    "Salario Neto": "salario_neto"
})

# ===============================
# 9) Calcular salario base = máximo entre básico y mínimo
# ===============================
tabla["salario_base"] = tabla[["salario_basico", "salario_minimo"]].max(axis=1)

# ===============================
# 10) Tabla final
# ===============================
resultado = tabla[["employee_name", "periodo", "salario_base", "salario_neto"]]

print("✅ Resumen mensual de salarios")
r =resultado.sort_values(["salario_neto"],ascending=False)


#aprox2min

✅ Resumen mensual de salarios


In [4]:
r[r["employee_name"].str.contains("FELIPE")].head(33)

rule_name,employee_name,periodo,salario_base,salario_neto
1972,OSCAR FELIPE RAMIREZ PARDO,2024-03,3750000.0,5524902.0
146,ANDRES FELIPE ORDOÑEZ GREGORY,2024-02,2600000.0,4930665.0
158,ANDRES FELIPE ORDOÑEZ GREGORY,2025-02,2846000.0,4882057.0
162,ANDRES FELIPE ORDOÑEZ GREGORY,2025-06,2847000.0,4073437.0
424,DANIEL FELIPE GONZALEZ LAGOS,2025-01,2846000.0,4044660.0
423,DANIEL FELIPE GONZALEZ LAGOS,2024-12,2811000.0,3812000.0
166,ANDRES FELIPE ORDOÑEZ GREGORY,2025-10,2847000.0,3802269.0
150,ANDRES FELIPE ORDOÑEZ GREGORY,2024-06,2600000.0,3799248.0
165,ANDRES FELIPE ORDOÑEZ GREGORY,2025-09,2847000.0,3684644.0
164,ANDRES FELIPE ORDOÑEZ GREGORY,2025-08,2847000.0,3655675.0
